In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
session = requests.Session()

params = {'username': "k33shen", 'password': "Qazwsx@1703", '_eventId': 'submit', 'submit':'LOGIN', 'lt': 'e1s1'}

In [3]:
def find_total_page_count(bsObj):
    """Returns the total number of postings"""
    posting_count = bsObj.find("div", {"id": "postingsTablePlaceholder"}).find("span", {"class": "badge badge-info"}).get_text().strip()
    return (int(posting_count) - 1) // 100 + 1 #because math module isn't found so math.ceil is unavailable

def find_action_value(bsObj):
    script = bsObj.find('div',{'class': 'tab-content'}).find('script')
    script = str(script)
    function_start_index = script.find('function loadPostingTable')
    first_apostrophe_index = script.find("'", function_start_index)
    second_apostrophe_index = script.find("'", first_apostrophe_index + 1)
    return script[first_apostrophe_index + 1:second_apostrophe_index]

In [6]:
with requests.Session() as s:

    #connection = create_database()
    s.get('https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm')
    p = s.post('https://cas.uwaterloo.ca/cas/login?service=https://waterlooworks.uwaterloo.ca/waterloo.htm', data=params)
    print('==================')

    #go to all postings

    postings_home_page = s.get('https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm')

    #get the value for 'Content-Disposition: form-data; name="action"'
    bsObj = BeautifulSoup(postings_home_page._content, "html.parser")
    a = bsObj.find("form", {"id": "widgetSearch"})
    #print(a)
    action_value = bsObj.find("form", {"id": "widgetSearch"}).find("input", {"name": "action"})['value']
    initial_search_action_value = bsObj.find("form", {"id": "widgetSearch"}).find("input", {"name": "initialSearchAction"})['value']
    
    #search data is gotten via the html info from Inspect elements
    search_data = {
    	'action': action_value,
        'initialSearchAction' : initial_search_action_value,
    	'filter': 'all',
    	'widget1Keyword': '',
    	'widget2Keyword': ''
    }

    files = {'': ('', '')}


    search_page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm', data=search_data, files=files)

    #####look for the action value in order to go to next page. It's inside
    '''
    function
    loadPostingTable(orderBy, access, customItem, callback, page)
    {
        var
    request = {
        action: '_-_-DtDt7fikcywBvXJ5_GeUVj5FoLjMzUnTyXWgiil8BBQ8Byq4neMkVyIpv2ETW2g6BMkCoBSAjurw07r5KrzL8hKieUeLQOeVo6IIf31ccKMHDBISGKNPo3--CDlArYy0RyzZlG_lKqUNlJAVNNiiWA6pqu1mFPyKBMbqXXSBP_st',
        orderBy: orderBy,
        postings: access,
        page: page,
        initialSearchAction: '',
        searchType: '',
        npfGroup: $('[name=npfGroup]').val(),
                   rand: Math.floor(Math.random() * 100000)
    };
    if (customItem)
    '''
    bsObj = BeautifulSoup(search_page._content, "html.parser")

    total_page_count = find_total_page_count(bsObj)
    action_value_100_postings = find_action_value(bsObj)
    
    total_page_count = 1 #REMOVE
    
    rows = []
    for page_count in range(1, total_page_count + 1): #stop at total_page_count + 1 because range is not inclusive
        print ('========================= page_count: ' + str(page_count) + '==========================================')
        data_100_postings = {
            'action': action_value_100_postings,
            'initialSearchAction' : initial_search_action_value,
            'sortDirection': 'Forward',
            'postings': 'infoForPostings',
            'page': str(page_count)
        }

        page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm', data=data_100_postings)
        bsObj = BeautifulSoup(page._content, "html.parser")
        tbody = ""
        try:
            tbody = bsObj.select("table#postingsTable > tbody")[0]
        except:
            print('=============== OH NO!! =================')
            print(page.text)

        postings = tbody.find_all('tr')
        #### Look at all jobs on page
        for i in range(len(postings)):

            tds = postings[i].find_all('td')
            #print("Row data: ", tds)
            id = tds[2].get_text() # used to be tds[1], now it's tds[2]. There are changes every once in a while.
            print(id)
            
            num_apps = tds[10].get_text()
            print('Num applications: ', num_apps)
            
            deadline = tds[11].get_text()
            print('Deadline: ', deadline)

            jsFunction = tds[3].find('a')['onclick'] # used to be tds[2], now tds[3]
            #print(jsFunction)

            # This is hard-coded and brittle but I want a quick fix
            # orbisApp.buildForm({'action':'THISISIT!',....})

            startIndex = 30
            endIndex = jsFunction.find("'", startIndex + 1)

            action_value = jsFunction[startIndex : endIndex]
            #print(action_value)

            '''
            #get initialSearchAction
            startIndex = endIndex + 25
            endIndex = jsFunction.find("'", startIndex+1)
            initial_search_action = jsFunction[startIndex : endIndex]
            print(initial_search_action)
            '''


            '''
            path = tds[2].find('a')['href']
            print(path)
            intermediate_page = s.get('https://waterlooworks.uwaterloo.ca/myAccount/hire-waterloo/full-time-jobs/jobs-postings.htm' + path) #to get action to go to real page
            print (intermediate_page)
            bsObj = BeautifulSoup(intermediate_page._content, "html.parser")
            print(bsObj)
            action_value = bsObj.find("form", {"id": "rform"}).find("input", {"name": "action"})['value']
            '''

            data = {
                'action': action_value,
                'initialSearchAction': initial_search_action_value,
                'searchType':'',
                'accessToPostings':'infoForPostings',
                'postingId': id,
                'npfGroup':'',
                'sortDirection': 'Reverse',
            }


            files = {'': ('', '')}
            posting_page = s.post('https://waterlooworks.uwaterloo.ca/myAccount/co-op/coop-postings.htm', data=data, files=files)
            bsObj = BeautifulSoup(posting_page._content, "html.parser")

            #finding company info, the top <h5> for some reason isn't being found. Maybe because of the ::before
            organization_name = ""
            division_name = ""

            try:
                org_table = bsObj.find_all('table')[-1]
                organization_name = (org_table.find_all('tr')[0]).find_all('td')[1].get_text().strip()
                division_name = (org_table.find_all('tr')[1]).find_all('td')[1].get_text().strip()
                #print(organization_name)
                
                app_delivery_table = bsObj.find_all('table')[-2]
                
                trs = bsObj.find('div', {'id': 'postingDiv'}).find('table').find('tbody').find_all('tr', recursive=False)
                app_table_trs = app_delivery_table.find('tbody').find_all('tr', recursive=False)

#                 info = {
# 	                'Organization:': organization_name,
# 	                'Division:': division_name,
# 	                'Work Term:' : '',
# 	                'Job Type:' : '',
# 	                'Job Title:' : '',
# 	                'Number of Job Openings:':'',
# 	                'Job Category (NOC):':'',
# 	                'Level:':'',
# 	                'Region:':'',
# 	                'Job - Address Line One:':'',
# 	                'Job - Address Line Two:':'',
# 	                'Job - City:':'',
# 	                'Job - Province / State:': '',
# 	                'Job - Postal Code / Zip Code (X#X #X#):': '',
# 	                'Job - Country:': '',
# 	                'Work Term Duration:': '',
# 	                'Job Summary:': '',
# 	                'Job Responsibilities:': '',
# 	                'Required Skills:': '',
# 	                'Targeted Degrees and Disciplines:':'',
# 	                'Transportation and Housing:': '',
# 	                'Compensation and Benefits Information:':'',
# 	                'Additional Information:': '',
# 	                'Additional Job Identifiers:':'',
# 	                'Special Job Requirements:':'',
# 	                'Other':''
# 	            }
                
                #FUll time job structure
                info = {
                    'Id': id,
                    'Number of Applications': num_apps,
                    'Application Deadline': deadline,
	                'Organization:': organization_name,
	                'Division:': division_name,
	                'Term Posted:' : '',
	                'Position Type:': '',
	                'Job Title:' : '',
	                'Number of Job Openings:':'',
	                'Job Category (NOC):':'',
                    'Level:':'',
	                'Region:':'',
                    'Job - Address Line One:':'',
                    'Job - City:':'',
                    'Job - Province / State:':'',
                    'Job - Postal Code / Zip Code (X#X #X#):':'',
                    'Job - Country:':'',
                    'Work Term Duration:':'',
	                'Start Date:': '',
	                'Job Summary:': '',
	                'Job Responsibilities:': '',
	                'Required Skills:': '',
	                'Targeted Degrees and Disciplines:': '',
                    'Transportation and Housing:': '',
	                'Compensation and Benefits Information:':'',
	                'Additional Information:': '',
	                'Additional Job Identifiers:':'',
	                'Special Job Requirements:':'',
                    'Additional Application Information:':'',
                    'Application Delivery:': '',
                    'If by eMail, send to:': '',
                    'Application Documents Required:':'',
                    'Application Method:':'',
	                'Other':''
	            }


                for tr in trs:
                    tds = tr.find_all('td')
                    type = tds[0].get_text().strip()
                    value = tds[1].get_text().strip()
                    if type in info:
                        info[type] = value
                    else:
                        info['Other'] = value
                        
                for tr in app_table_trs:
                    tds = tr.find_all('td')
                    type = tds[0].get_text().strip()
                    value = tds[1].get_text().strip()
                    if type in info:
                        info[type] = value
                    else:
                        info['Other'] = value                   

                #print("COLLECTED INFO")
                #print(info)
                
                #append to rows array
                rows.append(info)
                
                
            except:
                print("Something bad happened and this job was not stored")


========================= page_count: 1==========================================
130429
Num applications:  12
Deadline:  
								Sep 24, 2019
								
							
131650
Num applications:  2
Deadline:  
								Sep 24, 2019
								
							
131651
Num applications:  1
Deadline:  
								Sep 24, 2019
								
							
131652
Num applications:  5
Deadline:  
								Sep 24, 2019
								
							
127883
Num applications:  0
Deadline:  
								Sep 24, 2019
								
							
129933
Num applications:  26
Deadline:  
								Sep 24, 2019
								
							
133056
Num applications:  3
Deadline:  
								Sep 24, 2019
								
							
133055
Num applications:  8
Deadline:  
								Sep 24, 2019
								
							
130814
Num applications:  12
Deadline:  
								Sep 24, 2019
								
							
129865
Num applications:  1
Deadline:  
								Sep 24, 2019
								
							
131583
Num applications:  1
Deadline:  
								Sep 24, 2019
								
							
130396
Num applications:  4
Deadline:  
								Sep 24, 2

In [7]:
#save to pandas dataframe
df = pd.DataFrame.from_dict(rows, orient="columns")

In [8]:
len(rows)

99

In [9]:
pd.set_option('display.max_columns', None)  
df.head()

,Additional Application Information:,Additional Information:,Additional Job Identifiers:,Application Deadline,Application Delivery:,Application Documents Required:,Application Method:,Compensation and Benefits Information:,Division:,Id,"If by eMail, send to:",Job - Address Line One:,Job - City:,Job - Country:,Job - Postal Code / Zip Code (X#X #X#):,Job - Province / State:,Job Category (NOC):,Job Responsibilities:,Job Summary:,Job Title:,Level:,Number of Applications,Number of Job Openings:,Organization:,Other,Position Type:,Region:,Required Skills:,Special Job Requirements:,Start Date:,Targeted Degrees and Disciplines:,Term Posted:,Transportation and Housing:,Work Term Duration:
0,Let us know a little something about you! What...,,,"\r\n\t\t\t\t\t\t\t\tSep 24, 2019\r\n\t\t\t\t\t...",,"University of Waterloo Co-op Work History,Résu...",WaterlooWorks,,Divisional Office,130429,,49 Spadina Ave,Toronto,Canada,M5V 2J1,Ontario,2174 Computer Programmers and Interactive Medi...,- Implement or improve features for the 1Passw...,1Password is looking for promising new program...,Junior Developer,Junior\n\n\nIntermediate\n\n\nSenior,12,2,1Password,"Sep 24, 2019 09:00:00 AM",,ON - Toronto,What you bring to the table: \r\n- Confidence:...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,"The 1Password team works remotely, in one of o...",4 month work term
1,NOTE: All interview candidates are required t...,,,"\r\n\t\t\t\t\t\t\t\tSep 24, 2019\r\n\t\t\t\t\t...",,"University of Waterloo Co-op Work History,Résu...",WaterlooWorks,Salary will match the qualifications and exper...,Divisional Office,131650,,30 Kern Road,Toronto,Canada,M3B1T1,Ontario,2173 Software Engineers and Designers,Full stack development of YuJa.com multimedia ...,360 Education Labs is a premiere software/hard...,Software Developers,Junior\n\n\nIntermediate\n\n\nSenior,2,5,360 Education Labs Inc,"Sep 24, 2019 09:00:00 AM",,ON - Toronto,Strong software design and development skills ...,8 month consecutive term preferred OR 2 altern...,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,The job location is close to a TTC subway stop...,2 work term commitment preferred
2,NOTE: All interview candidates are required t...,,,"\r\n\t\t\t\t\t\t\t\tSep 24, 2019\r\n\t\t\t\t\t...",,"University of Waterloo Co-op Work History,Résu...",WaterlooWorks,Salary will match the qualifications and exper...,Divisional Office,131651,,30 Kern Road,Toronto,Canada,M3B1T1,Ontario,2173 Software Engineers and Designers,Software testing and quality assurance of soft...,360 Education Labs is a premiere software/hard...,Software Developer - Testing and Operations,Junior\n\n\nIntermediate,1,4,360 Education Labs Inc,"Sep 24, 2019 09:00:00 AM",,ON - Toronto,Software design and development skills with J...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,Easy access via TTC buses and Subway.,4 month work term
3,Career Development and Training: Some on the j...,,,"\r\n\t\t\t\t\t\t\t\tSep 24, 2019\r\n\t\t\t\t\t...",,"University of Waterloo Co-op Work History,Résu...",WaterlooWorks,Salary is based on the salary survey tables pr...,Divisional Office,131652,,30 Kern Road,Toronto,Canada,M3B 1T1,Ontario,4143 Educational Counsellors,Providing sample tutoring sessions to prospect...,360 Education Labs is a premiere software/hard...,Science and Math Online Expert,Junior\n\n\nIntermediate,5,6,360 Education Labs Inc,"Sep 24, 2019 09:00:00 AM",,ON - Toronto,Some prior experience in customer service or s...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,Our office is conveniently located close to Sh...,4 month work term
4,,,,"\r\n\t\t\t\t\t\t\t\tSep 24, 2019\r\n\t\t\t\t\t...",,"University of Waterloo Co-op Work History,Cove...",WaterlooWorks,Expect all the comforts that come with the sta...,Divisional Office,127883,,55 Water Street,Vancouver,Canada,V6B 1A1,British Columbia,2175 Web Designers and Developers,Responsibilities:\r\n- Build new features and ...,"Who we are\r\n7Geese is a fun, ambitious start...",Developer

In [ ]:
print(df.columns)

In [ ]:
import fastparquet

In [ ]:
#df.to_hdf('data.h5', key="s")
df.to_parquet('coop-F2019.parquet', compression="gzip", engine='fastparquet')

In [ ]:
test = pd.read_parquet("coop-F2019.parquet", engine="fastparquet")

In [ ]:
len(test)

In [16]:
test.head()

,Additional Application Information:,Additional Information:,Additional Job Identifiers:,Application Deadline,Application Delivery:,Application Documents Required:,Application Method:,Compensation and Benefits Information:,Division:,Id,"If by eMail, send to:",Job - Address Line One:,Job - City:,Job - Country:,Job - Postal Code / Zip Code (X#X #X#):,Job - Province / State:,Job Category (NOC):,Job Responsibilities:,Job Summary:,Job Title:,Level:,Number of Applications,Number of Job Openings:,Organization:,Other,Position Type:,Region:,Required Skills:,Special Job Requirements:,Start Date:,Targeted Degrees and Disciplines:,Term Posted:,Transportation and Housing:,Work Term Duration:
0,Let us know a little something about you! What...,,,Toronto,,"University of Waterloo Co-op Work History,Résu...",WaterlooWorks,,Divisional Office,130429,,49 Spadina Ave,Toronto,Canada,M5V 2J1,Ontario,2174 Computer Programmers and Interactive Medi...,- Implement or improve features for the 1Passw...,1Password is looking for promising new program...,Junior Developer,Junior\n\n\nIntermediate\n\n\nSenior,3,2,1Password,"Sep 24, 2019 09:00:00 AM",,ON - Toronto,What you bring to the table: \r\n- Confidence:...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,"The 1Password team works remotely, in one of o...",4 month work term
1,NOTE: All interview candidates are required t...,,,Toronto,,"University of Waterloo Co-op Work History,Résu...",WaterlooWorks,Salary will match the qualifications and exper...,Divisional Office,131650,,30 Kern Road,Toronto,Canada,M3B1T1,Ontario,2173 Software Engineers and Designers,Full stack development of YuJa.com multimedia ...,360 Education Labs is a premiere software/hard...,Software Developers,Junior\n\n\nIntermediate\n\n\nSenior,1,5,360 Education Labs Inc,"Sep 24, 2019 09:00:00 AM",,ON - Toronto,Strong software design and development skills ...,8 month consecutive term preferred OR 2 altern...,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,The job location is close to a TTC subway stop...,2 work term commitment preferred
2,NOTE: All interview candidates are required t...,,,Toronto,,"University of Waterloo Co-op Work History,Résu...",WaterlooWorks,Salary will match the qualifications and exper...,Divisional Office,131651,,30 Kern Road,Toronto,Canada,M3B1T1,Ontario,2173 Software Engineers and Designers,Software testing and quality assurance of soft...,360 Education Labs is a premiere software/hard...,Software Developer - Testing and Operations,Junior\n\n\nIntermediate,1,4,360 Education Labs Inc,"Sep 24, 2019 09:00:00 AM",,ON - Toronto,Software design and development skills with J...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,Easy access via TTC buses and Subway.,4 month work term
3,Career Development and Training: Some on the j...,,,Toronto,,"University of Waterloo Co-op Work History,Résu...",WaterlooWorks,Salary is based on the salary survey tables pr...,Divisional Office,131652,,30 Kern Road,Toronto,Canada,M3B 1T1,Ontario,4143 Educational Counsellors,Providing sample tutoring sessions to prospect...,360 Education Labs is a premiere software/hard...,Science and Math Online Expert,Junior\n\n\nIntermediate,2,6,360 Education Labs Inc,"Sep 24, 2019 09:00:00 AM",,ON - Toronto,Some prior experience in customer service or s...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,Our office is conveniently located close to Sh...,4 month work term
4,,,,Vancouver,,"University of Waterloo Co-op Work History,Cove...",WaterlooWorks,Expect all the comforts that come with the sta...,Divisional Office,127883,,55 Water Street,Vancouver,Canada,V6B 1A1,British Columbia,2175 Web Designers and Developers,Responsibilities:\r\n- Build new features and ...,"Who we are\r\n7Geese is a fun, ambitious start...",Developer (Startup),Intermediate\n\n\nSenior,0,1,7Geese,"Sep 24, 2019 09:00:00 AM",,Western Canada,Skills and qualifications\r\n- Desire to solve...,,,Targeted Clusters\r\n\t\t\t\n\r\n\t\t\r\n\t\t\...,,We will pay airfare to/fro